# Exploring Clojure and Java interop

## Calling Java from Clojure

### Importing Java classes into Clojure

In [1]:
; The general form of import statements is as follows

"""
(import & import-symbols-or-lists)
"""

""

In [2]:
; Importing 2 java classes individually

;The quote (') reader macro instructs the runtime not to evaluuate the symbol
(import 'java.util.Date 'java.text.SimpleDateFormat) 

java.text.SimpleDateFormat

In [3]:
; Importing 2 java classes as a sequence

(import '[java.util Date Set])

java.util.Set

In [4]:
; Using the :import keyword to import classes in a namespace

(ns com.clojureinaction.book (:import (java.util Set Date)))

nil

In [5]:
(ns user)

nil

### Creating instances

In [6]:
; Using the new keyword to instantiate a class (like in Java)

(import '(java.text SimpleDateFormat))

(def sdf (new SimpleDateFormat "yyyy-MM-dd"))

#'user/sdf

In [7]:
; Using a trailing dot to instantiate a class 

(def sdf (SimpleDateFormat. "yyyy-MM-dd"))


#'user/sdf

###  Accessing methods and fields

In [8]:
; Using a leading dot to access an instance method

(defn date-from-date-string [date-string]
    (let [sdf (SimpleDateFormat. "yyyy-MM-dd")] 
        (.parse sdf date-string)))

#'user/date-from-date-string

In [9]:
; Using a slash to access a static method

(Long/parseLong "12321") ; The syntax is (Classname/staticMethod args*)


12321

In [10]:
; Using a slash to access a static field

(import '(java.util Calendar))

(Calendar/JANUARY)

0

### Macros and the dot special form

In [11]:
; General form of calling static methods

"""
(. ClassnameSymbol methodSymbol args*)
"""

; Example

(. System getenv "PATH")

"/home/william/.vscode-server/bin/899d46d82c4c95423fb7e10e68eba52050e30ba3/bin:/home/william/.local/bin:/home/william/miniconda3/envs/clojupyter/bin:/home/william/miniconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/Python27:/mnt/c/Python27/Scripts:/mnt/c/Windows/system32:/mnt/c/Windows:/mnt/c/Windows/System32/Wbem:/mnt/c/Windows/System32/WindowsPowerShell/v1.0:/mnt/c/Windows/System32/OpenSSH:/mnt/c/Users/Acer/AppData/Local/Microsoft/WindowsApps:/mnt/c/Users/Acer/AppData/Local/Programs/Microsoft VS Code/bin:/snap/bin"

In [12]:
; General form of calling instance methods

"""
(. instanceExpr methodSymbol args*)
"""

; Example

(import '(java.util Random))


(def rnd (Random.))
(. rnd nextInt 10)


8

In [13]:
; General form of calling static and instance fields

"""
(. ClassnameSymbol memberSymbol)

(. instanceExpr memberSymbol)
"""

; Example

(. Calendar DECEMBER)

11

### The Dot-Dot macro

Using the '.' macro to chain Java method calls (hard to read)

In [14]:
(import '(java.util Calendar TimeZone))

(. (. (Calendar/getInstance) (getTimeZone)) (getDisplayName))


"Paraguay Time"

Using the '..' macro to chain method calls (easier to read)

In [15]:
(.. (Calendar/getInstance) (getTimeZone) (getDisplayName))


"Paraguay Time"

### The Doto macro

In [16]:
; Applying a method repeteadly to a single Java object (note the code duplication)

(import '(java.util Calendar))

(defn the-past-midnight-1 []
    (let [calendar-obj (Calendar/getInstance)]
        (.set calendar-obj Calendar/AM_PM Calendar/AM)
        (.set calendar-obj Calendar/HOUR 0)
        (.set calendar-obj Calendar/MINUTE 0)
        (.set calendar-obj Calendar/SECOND 0)
        (.set calendar-obj Calendar/MILLISECOND 0)
        (.getTime calendar-obj)))


#'user/the-past-midnight-1

In [17]:
; Using the doto macro to apply a method repeteadly to a single Java object 
; (the code duplication was removed)

(defn the-past-midnight-2 []
    (let [calendar-obj (Calendar/getInstance)]
        (doto calendar-obj
            (.set Calendar/AM_PM Calendar/AM)
            (.set Calendar/HOUR 0)
            (.set Calendar/MINUTE 0)
            (.set Calendar/SECOND 0)
            (.set Calendar/MILLISECOND 0))
        (.getTime calendar-obj)))


#'user/the-past-midnight-2

### The memfn macro

In [18]:
; Using a Java method as a normal function

(map (fn [x] (.getBytes x)) ["amit" "rob" "kyle"])

(#object["[B" 0x7c1581dc "[B@7c1581dc"] #object["[B" 0x1a76d8c "[B@1a76d8c"] #object["[B" 0x1783db6d "[B@1783db6d"])

In [19]:
; Using a Java method as an anonymous function

(map #(.getBytes %) ["amit" "rob" "kyle"])

(#object["[B" 0x35a1997a "[B@35a1997a"] #object["[B" 0x2241d869 "[B@2241d869"] #object["[B" 0x47a4d528 "[B@47a4d528"])

In [20]:
; Using the memfn macro instead of the anonymous function

(map (memfn getBytes) ["amit" "rob" "kyle"])

(#object["[B" 0x55a00c6e "[B@55a00c6e"] #object["[B" 0x46bdd49e "[B@46bdd49e"] #object["[B" 0x142bad96 "[B@142bad96"])

In [21]:
; Calling a Java method without type hints

(.subSequence "Clojure" 2 5)

"oju"

In [22]:
; Calling a Java method with type hints

((memfn ^String subSequence ^Long start ^Long end) "Clojure" 2 5)

"oju"

### The bean macro

In [23]:
; Converting Java bean objects to immutable Clojure maps

(import '[java.util Calendar])

(bean (Calendar/getInstance))


{:weeksInWeekYear 53, :timeZone #object[sun.util.calendar.ZoneInfo 0x450f1b06 "sun.util.calendar.ZoneInfo[id=\"America/Asuncion\",offset=-14400000,dstSavings=3600000,useDaylight=true,transitions=129,lastRule=java.util.SimpleTimeZone[id=America/Asuncion,offset=-14400000,dstSavings=3600000,useDaylight=true,startYear=0,startMode=3,startMonth=9,startDay=1,startDayOfWeek=1,startTime=0,startTimeMode=0,endMode=3,endMonth=2,endDay=22,endDayOfWeek=1,endTime=0,endTimeMode=0]]"], :weekDateSupported true, :weekYear 2022, :lenient true, :time #inst "2022-01-04T20:49:42.311-00:00", :calendarType "gregory", :timeInMillis 1641329382311, :class java.util.GregorianCalendar, :firstDayOfWeek 1, :gregorianChange #inst "1582-10-15T00:00:00.000-00:00", :minimalDaysInFirstWeek 1}

Working with Java arrays

In [24]:
(def tokens (.split "clojure.in.action" "\\."))

#'user/tokens

###  Implementing interfaces and extending classes

The proxy macro

In [25]:
; Implementing the MouseAdapter class with proxy

(import 'java.awt.event.MouseAdapter)


(proxy [MouseAdapter] []
    (mousePressed [event]
        (println "Hey!"))) 

#object[user.proxy$java.awt.event.MouseAdapter$ff19274a 0x3c76d7ff "user.proxy$java.awt.event.MouseAdapter$ff19274a@3c76d7ff"]

In [26]:
; The general form of the proxy macro is as follows

"""
(proxy [class-and-interfaces] [args] fs+)
"""


""

The reify macro

In [27]:
; Creating an instance of Java’s FileFilter interface

(reify java.io.FileFilter
    (accept [this f]
        (.isDirectory f)))

#object[user$eval4187$reify__4188 0x277a65d6 "user$eval4187$reify__4188@277a65d6"]

## Compiling Clojure code to Java bytecode

The first dierctory structure of the project

```
root
    classes
    src
        com
            curry
                utils
                    calculators.clj
```

In [2]:
; Contents of the calculators.clj file

"""
(ns com.curry.utils.calculators (:gen-class))

(defn present-value [data]
    (println \"calculating present value...\")
"""

""

In [ ]:
; Compiling the defined namespace

(compile 'com.curry.utils.calculators)

The new directory structure of the project

```
root
    classes
    src
        com
            curry
                utils
                    calculators.clj
                    calc
                        dcf.clj
                        fcf.clj
```

In [ ]:
; Contents of dcf.clj

"""
(in-ns 'com.curry.utils.calculators)

(defn discounted-cash-flow [data]
    (println \"calculating discounted cash flow...\"))
"""

In [ ]:
; Contents of fcf.clj

"""
(in-ns 'com.curry.utils.calculators)

(defn free-cash-flow [data]
    (println \"calculating free cash flow...\"))
"""

In [3]:
; The new contents of calculators.clj

"""
(ns com.curry.utils.calculators (:gen-class))
(load \"calc/fcf\")
(load \"calc/dcf\")


(defn present-value [data]
    (println \"calculating present value...\"))
"""

""

### Creating Java classes and interfaces using gen-class and gen-interface

In [ ]:
; An abstract Java class that will be used to illustrate 
; how gen-class works

"""
package com.gentest;


public abstract class AbstractJavaClass {
    public AbstractJavaClass(String a, String b) {
        System.out.println(\"Constructor: a, b\");
    }
    public AbstractJavaClass(String a) {
        System.out.println(\"Constructor: a\");
    }
    public abstract String getCurrentStatus();
        public String getSecret() {
            return \"The Secret\";
        }
}
"""


In [5]:
; Using the last java AbstractJavaClass inside clojure code

"""
(ns com.gentest.gen-clojure
    (:import (com.gentest AbstractJavaClass))
    (:gen-class
     :name com.gentest.ConcreteClojureClass
     :extends com.gentest.AbstractJavaClass
     :constructors {[String] [String]
                    [String String] [String String]}
     :implements [Runnable]
     :init initialize
     :state localState
     :methods [[stateValue [] String]]))


(defn -initialize
    ([s1]
        (println \"Init value:\" s1)
        [[s1 \"default\"] (ref s1)])
    ([s1 s2]
        (println \"Init values:\" s1 \",\" s2)
        [[s1 s2] (ref s2)]))

(defn -getCurrentStatus [this]
    \"getCurrentStatus from - com.gentest.ConcreteClojureClass\")

(defn -stateValue [this]
    @(.localState this))

(defn -run [this]
    (println \"In run!\")
    (println \"I'm a\" (class this))
    (dosync (ref-set (.localState this) \"GO\")))

(defn -main []
    (let [g (new com.gentest.ConcreteClojureClass \"READY\")]
        (println (.getCurrentStatus g))
        (println (.getSecret g))
        (println (.stateValue g)))
    (let [g (new com.gentest.ConcreteClojureClass \"READY\" \"SET\")]
        (println (.stateValue g))
        (.start (Thread. g))
        (Thread/sleep 1000)
        (println (.stateValue g))))
"""


""

In [ ]:
; To compile and test the last code, execute the following commands in the REPL

!compile 'com.gentest.gen-clojure
!java com.gentest.ConcreteClojureClass


Leiningen project file for ConcreteClojureClass

In [7]:
"""
(defproject gentest \"0.1.0\"
    :dependencies [[org.clojure/clojure \"1.6.0\"]]
    
    ; Place our \"AbstractJavaClass.java\" and \"gen-clojure.clj\" files under
    ; the src/com/gentest directory.
    :source-paths [\"src\"]
    :java-source-paths [\"src\"]
    
    ; :aot is a list of clojure namespaces to compile.
    :aot [com.gentest.gen-clojure]
    
    ; This is the java class \"lein run\" should execute.
    :main com.gentest.ConcreteClojureClass)
"""


""

## Calling Clojure from Java

In [8]:
;  Clojure function, defined in the clj.script.examples namespace
(ns clj.script.examples)


(defn print-report [user-name]
    (println "Report for:" user-name) 10)


#'clj.script.examples/print-report

In [ ]:
; Using the last function in a Java code

"""
import clojure.lang.RT;
import clojure.lang.Var;


public class Driver {
    public static void main(String[] args) throws Exception {
        RT.loadResourceScript(\"clojure_script.clj\");
        Var report = RT.var(\"clj.script.examples\", \"print-report\");
        Integer result = (Integer) report.invoke(\"Siva\");
        System.out.println(\"Result: \" + result);
    }
}
"""
